In [1]:
import pandas as pd
benign_df = pd.read_csv("benign_domain_list.csv")
malicious_df = pd.read_csv("Malicious_URLs.csv")
mixed_df = pd.read_csv("mixed_domain_list.csv")
tranco_df = pd.read_csv("tranco_full_list_for_ranking.csv")